# Generic Rule Induction Notebook

Continuoulsy refined.

In [1]:
#%store -z acc_list 
#%store -z prec_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [2]:
from config import config_dict
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG-I4"]
print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: miniloan


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import time
import warnings
import re


if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
elif CONFIG['ALGO'] == 'BRCG':
    from aix360.algorithms.rbm import BooleanRuleCG # BRCGExplainer
elif CONFIG['ALGO'] == 'CORELS':
    from corels import *
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    import wittgenstein as lw
elif CONFIG['ALGO'] == 'GLRM':
    from aix360.algorithms.rbm import GLRMExplainer, LinearRuleRegression

# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


Importing dev version v0.982 of RIPPER


### Data

In [4]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   creditScore          100000 non-null  float64
 1   income               100000 non-null  float64
 2   loanAmount           100000 non-null  float64
 3   monthDuration        100000 non-null  float64
 4   rate                 100000 non-null  float64
 5   approval             100000 non-null  bool   
 6   yearlyReimbursement  100000 non-null  float64
dtypes: bool(1), float64(6)
memory usage: 4.7 MB


True     78046
False    21954
Name: approval, dtype: int64

In [5]:
# One Hot
# Helper Function
if CONFIG["ONEHOT"] == True:
    from collections import defaultdict
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder
    categorial_feat = df.select_dtypes(include=['object']).columns
    numercial_feat = df.select_dtypes(include=['int64', 'float64']).columns

    def one_hot_encode_category(df, categorial_index):
        d = defaultdict(LabelEncoder)
        # Encoding
        lecatdata = df[categorial_index].apply(lambda x: d[x.name].fit_transform(x))
        for x in range(len(categorial_index)):
            print(categorial_index[x],": ", df[categorial_index[x]].unique())
            print(categorial_index[x],": ", lecatdata[categorial_index[x]].unique())
        #One hot encoding with dummy variable
        dummyvars = pd.get_dummies(df[categorial_index])

        return dummyvars

    dummyvars =one_hot_encode_category(df,categorial_feat )
    df = pd.concat([df[numercial_feat], dummyvars], axis = 1)
    df

In [6]:

# undersampling for config 2
''' 
under_sampling_fraud = df[df["Class"] == 1]
under_size = under_sampling_fraud.size
under_sampling_legit = (df[df["Class"] == 0]).sample(n=10000, random_state = 42)
df = pd.concat([under_sampling_legit, under_sampling_fraud])'''

' \nunder_sampling_fraud = df[df["Class"] == 1]\nunder_size = under_sampling_fraud.size\nunder_sampling_legit = (df[df["Class"] == 0]).sample(n=10000, random_state = 42)\ndf = pd.concat([under_sampling_legit, under_sampling_fraud])'

In [7]:
CONFIG['POS_CLASS']

False

In [8]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

True     78046
False    21954
Name: approval, dtype: int64


### Train, Test Split

In [9]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (70000, 6) (70000,)
Test: (30000, 6) (30000,)


### Reference Performance

In [10]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [11]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 76513 to 15795
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   creditScore          70000 non-null  float64
 1   income               70000 non-null  float64
 2   loanAmount           70000 non-null  float64
 3   monthDuration        70000 non-null  float64
 4   rate                 70000 non-null  float64
 5   yearlyReimbursement  70000 non-null  float64
dtypes: float64(6)
memory usage: 3.7 MB


,creditScore,income,loanAmount,monthDuration,rate,yearlyReimbursement
76513,323.0,142660.0,1371289.0,155.0,0.057102,150327.0
60406,781.0,138610.0,88183.0,286.0,0.049289,6296.0
27322,458.0,190396.0,1073466.0,150.0,0.057622,120683.0
53699,569.0,36948.0,817845.0,359.0,0.050611,53114.0
65412,633.0,60520.0,1113649.0,289.0,0.037175,70057.0


### Rule Induction

In [12]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                      # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for RIPPER
set to True
Training time: 76.98400521278381


In [13]:
CONFIG['POS_CLASS']

False

### Evaluation

In [14]:
acc_list = []
prec_list = []
%store -r acc_list
%store -r prec_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    prec_list.append(precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    %store acc_list
    %store prec_list
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.9954333333333333
Balanced accuracy: 0.991695265503403
Precision: 0.9939981532779316
Recall: 0.9850541406130853
F1 0.9895059364228265
ConfusionMatrix [[ 6459    98]
 [   39 23404]]
F-2 0.9868300433905763
Mathhews 0.9866041622720696
Stored 'acc_list' (list)
Stored 'prec_list' (list)


In [15]:
y_test.value_counts()

True     23443
False     6557
Name: approval, dtype: int64

In [16]:
CONFIG['POS_CLASS']

False

In [17]:
CONFIG['BINARIZER']

'NATIVE'

In [18]:
CONFIG['ALGO']

'RIPPER'

In [19]:
df[CONFIG["TARGET_LABEL"]].value_counts()

True     78046
False    21954
Name: approval, dtype: int64

In [20]:
#XGboost for Binary Classification

    
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('ConfusionMatrix', confusion_matrix(y_test, preds))
    print('Mathhews', matthews_corrcoef(y_test, preds))
   

/Users/hvo/opt/miniconda3/envs/aix360iripper2/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:47:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.9982
Balanced accuracy: 0.997365266751356
Precision: 0.9958822632301357
Recall: 0.9958822632301357
F-2 0.9958822632301357
ConfusionMatrix [[ 6530    27]
 [   27 23416]]
Mathhews 0.9947305335027118


In [21]:
CONFIG['POS_CLASS']

False

In [22]:
präds = []
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    #print(estimator.rule_list_to_pretty_string())
    # Get predicates
    for i in range(len(estimator.rule_map[1])):
        print(len(estimator.rule_map[1][i]))
        präds.append(len(estimator.rule_map[1][i]))
    print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    # Get predicates
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        präds.append(len(estimator.rl().rules[i]["antecedents"]))
        print(f"Antecedents Length Rule {i}:" , an)
    print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Rule count: 30
Rule set:


KeyError: 1

In [ ]:
len(estimator.rule_map[0][3])